In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
import scipy.stats as st
import os.path
from os import path
import json

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_players_Titled = "output_data/players_Titled.csv"
output_data_players_RU =  "output_data/players_RU.csv"

titled_players_type = ["GM","WGM","IM","WIM","FM","WFM","NM","WNM","CM","WCM"]

# Range of latitudes and longitudes
# lat_range = (-90, 90)
# lng_range = (-180, 180)

In [12]:
# ***********************************
#
# LIMIT run once per 24 hours to get
# username list by country
#
# ***********************************

# country code # countries list resource https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2
country = "RU" # RUSSIA = RU

query_url = f"https://api.chess.com/pub/country/{country}/players"

print(f"Fetching player list for country {country}")

# Get player usernames for country
username_response = requests.get(query_url)
username_json = username_response.json()
username_json


Fetching player list for country RU


{'players': ['-aqe-',
  '0-0-0iscool',
  '0-bit-0',
  '00001111romeo',
  '0001a',
  '000325',
  '000999000goa',
  '000evgen',
  '000jacob000',
  '000shark',
  '00130712345',
  '001kostroma',
  '001mann',
  '001mexak',
  '001r78',
  '001riv',
  '001safa001',
  '001sv',
  '003322',
  '007bav',
  '009177',
  '009ed',
  '00bek00',
  '00count00',
  '00danil00',
  '00fischer',
  '00human0',
  '00malex',
  '00osm00',
  '00sophia00',
  '00sugarbear00',
  '00tupaya00',
  '00vladimir_poutine00',
  '00x19',
  '01arsenii',
  '01crimea2020',
  '01e9',
  '01mara',
  '01mlk10',
  '01roman',
  '02004',
  '02021971',
  '02022020fa',
  '02110',
  '02200',
  '02klemento02',
  '02martin02',
  '030108wqaswq',
  '03031966mikle0303196',
  '03031972',
  '030362',
  '033fishera',
  '03julia',
  '03lav03',
  '04061957alex',
  '04101981',
  '044btatik',
  '05012017',
  '050789',
  '05111977',
  '051973',
  '0555544111',
  '05adam1',
  '05hamimarkel05',
  '05omar05',
  '05oven',
  '05th',
  '061dobermann',
  '064

In [23]:
usernames = []
for username in username_json["players"]:
    usernames.append(username)
    
player_RU_dict = {    
    "username":usernames
}
chessplayer_RU_df = pd.DataFrame(player_RU_dict)
chessplayer_RU_df.to_csv(output_data_players_RU, index=True,index_label="player_index")  

In [24]:
chessplayer_RU_df

,username
0,-aqe-
1,0-0-0iscool
2,0-bit-0
3,00001111romeo
4,0001a
...,...
105844,z_v_l
105845,z_z_1
105846,_catharsis_
105847,_insomniac_


In [25]:
# count number of usernames
len(usernames)

105849

In [26]:
def get_titledplayers_api(playertypes):
    url = "https://api.chess.com/pub/titled/"
    
    i = 0        
    players_data = []
    
    for playertype in playertypes:
        print(f"Index {str(i)} fetching player list for titled player type {playertype}")
    
        # Build query URL
        query_url = f"{url}{playertype}"

        # increment index counter
        i += 1

        # Get weather data    
        playertype_response = requests.get(query_url)
        playertype_json = playertype_response.json()
        players_data.append(playertype_json)        
        time.sleep(1)    
        
    return players_data

In [27]:
def get_playerdata_api(player_usernames):
    url = "https://api.chess.com/pub/player/"
    
    i = 0        
    player_data = []
    
    for player_username in player_usernames:
        print(f"Index {str(i)} fetching player data for player {player_username}")
    
        # Build query URL
        query_url = f"{url}{player_username}"

        # increment index counter
        i += 1

        # Get player data    
        try:
            player_response = requests.get(query_url)
            player_json = player_response.json()
            player_data.append(player_json)        
            time.sleep(1)    
        except:
            print("unexpected error")
        
        # if i > 10:
        #    break
    return player_data

In [19]:
playerslist_data = get_titledplayers_api(titled_players_type)

Index 0 fetching player list for titled player type GM
Index 1 fetching player list for titled player type WGM
Index 2 fetching player list for titled player type IM
Index 3 fetching player list for titled player type WIM
Index 4 fetching player list for titled player type FM
Index 5 fetching player list for titled player type WFM
Index 6 fetching player list for titled player type NM
Index 7 fetching player list for titled player type WNM
Index 8 fetching player list for titled player type CM
Index 9 fetching player list for titled player type WCM


In [20]:
playerlist_index = 0
i = 0

player_usernames = []

for playerlist in playerslist_data:    
    for player in playerlist["players"]:        
        player_usernames.append(player)
        
    playerlist_index += 1
    

```{
  "avatar": "https://images.chesscomfiles.com/uploads/v1/user/14775486.9072453d.200x200o.50bb2d10a7d5.jpeg",
  "player_id": 14775486,
  "@id": "https://api.chess.com/pub/player/140mariam",
  "url": "https://www.chess.com/member/140mariam",
  "name": "Mariam Avetisyan",
  "username": "140mariam",
  "title": "WFM",
  "followers": 56,
  "country": "https://api.chess.com/pub/country/AM",
  "last_online": 1609780423,
  "joined": 1385829847,
  "status": "premium",
  "is_streamer": false
}
```

In [21]:
len(player_usernames)

9302

In [29]:
# check if output_data_file exists already, if it doesn't exist, do api call to get weather data
if (path.exists(output_data_players)):
    # ****************
    # pull prevously generated and saved output data from csv
    chessplayer_df = pd.read_csv(output_data_players)
else:    
    # run api calls for each chess.com username
    # this step is skipped if the output_data_file already exists
    # !!!!! DELETE CSV OUTPUT FILE to generate new playerdata    
    playerdata = get_playerdata_api(player_usernames)
    
    player_id = []
    username = []
    title = []
    followers = []
    country = []
    location = []
    last_online = []
    joined = []
    status = []
    is_streamer = []

    i = 0

    for player in playerdata: 
        try:
            un = player["username"]
            print(f"Index {i} processing player {un}")
            player_id.append(player["player_id"])
            username.append(player["username"])
            title.append(player["title"])
            followers.append(player["followers"])
            country.append(player["country"].replace("https://api.chess.com/pub/country/","")),
            if "location" in player.keys():
                location.append(player["location"])        
            else:
                location.append("")                
            last_online.append(player["last_online"])
            joined.append(player["joined"])
            status.append(player["status"])
            is_streamer.append(player["is_streamer"])
        except:
            print("unexpected error")

        i += 1 
        
    player_dict = {
        "player_id":player_id,
        "username":username,
        "title":title,
        "followers":followers,
        "country":country,
        "location":location,
        "last_online":last_online,
        "joined":joined,
        "status":status,
        "is_streamer":is_streamer
    }
    chessplayer_df = pd.DataFrame(player_dict)

    chessplayer_df.to_csv(output_data_players, index=True,index_label="player_index")  
    
    

In [30]:
chessplayer_df

,player_index,player_id,username,title,followers,country,location,last_online,joined,status,is_streamer,last_online_converted,joined_converted
0,0,18800602,123lt,GM,32,CN,NaN,1608465021,1410059361,premium,False,12/20/2020,9/7/2014
1,1,29499974,124chess,GM,79,RU,Красноярск,1609836300,1471316272,premium,False,1/5/2021,8/16/2016
2,2,30610578,1977ivan,GM,95,RS,Belgrade,1609872690,1477565847,premium,False,1/5/2021,10/27/2016
3,3,13013662,1stsecond,GM,734,LV,Riga,1609536323,1375617889,premium,False,1/1/2021,8/4/2013
4,4,39230304,2nd_life,GM,22,CH,Zurich,1609876221,1507311109,premium,False,1/5/2021,10/6/2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9295,9295,29980036,yashami,WCM,21,NP,Kathmandu,1594232302,1474135436,premium,False,7/8/2020,9/17/2016
9296,9296,67624176,yoriiallam,WCM,10,EG,NaN,1609501122,1573057158,premium,False,1/1/2021,11/6/2019
9297,9297,86574494,zaineb218,WCM,3,TN,Tunisie,1609446669,1594630390,premium,False,12/31/2020,7/13/2020
9298,9298,85888234,zarinur,WCM,15,KZ,NaN,1609904817,1594013985,premium,False,1/6/2021,7/6/2020
